<a href="https://colab.research.google.com/github/iron59zug/YFapi/blob/master/api_iex_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
# calling to API by a request w/ interpolated values
# IEX Cloud API not free of charge
# video course from freeCodeCamp (4 dec 2020) https://www.youtube.com/watch?v=xfzGZB4HhEE
# model file

import math
import numpy as np # coded in C, example of Python as glue for other faster languages
import pandas as pd # to treat tabular data eg from csv
import requests # for http communication
#import XlsxWriter # save output in excel not working in colab? 

SyntaxError: ignored

In [19]:
a1 = 'MSFT' # 1975
a2 = 'APPL' # 1976
a3 = 'AMZN' # 1994
a4 = 'GOOGL' # 1998; Alphabet's A share has 1 vote; C share "GOOG" has no vote
a5 = 'FB' # 2004

ticker = a1 # choose one asset name that will pass to ticker, or:
tickers = [a1, a2, a3, a4, a5] # use a list of tickers
tstring = ','.join(tickers) # serialize list into string

tstring

'MSFT,APPL,AMZN,GOOGL,FB'

In [20]:
endpoint = 'chart'
period = '1y'
KEY = 'Tpk_059b97af715d417d9f49f50b51b1c448' # free IEX Cloud API key for testing
''' for API key stored in key.py file
    (IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448')
from key import IEX_CLOUD_API_TOKEN
'''

# request = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={string}&types={endpoint}&range={period}&token={KEY}' # non-free api
request = f'https://sandbox.iexapis.com/stable/stock/{a1}/quote?token={KEY}' # free, not real data #fstring
df = requests.get(request) # get/post/put/delete
print (df.status_code == 200) # check url if 404

True


In [21]:
df = requests.get(request).json()
#df # testing whether receive info eg. vol/hogh/low/close...

In [22]:
close = df['close']
mc = df['marketCap']
close,mc

(None, 1860847759019)

In [23]:
ct = ['ticker', 'close price', 'market cap', 'buy qty']
final_df = pd.DataFrame([[1,2,3,4]], columns = ct)
final_df

,ticker,close price,market cap,buy qty
0,1,2,3,4


In [24]:
final_df.append(
    pd.Series(
        [ticker,
         close,
         mc,
         'n/a'
        ],
    index = ct
    ),
    ignore_index=True # necessary whenever use append???
)

,ticker,close price,market cap,buy qty
0,1,2,3,4
1,MSFT,None,1860847759019,n/a


In [25]:
# using dynamic input from certain API would be ideal; using static list for now
from google.colab import drive
drive.mount('/content/drive')

# load static ticker list
tlist = pd.read_csv('/content/drive/MyDrive/sp_500_stocks.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
final_df = pd.DataFrame(columns = ct)
for ticker in tlist['Ticker'][:5]: # check first 5 tickers only
  request = f'https://sandbox.iexapis.com/stable/stock/{ticker}/quote?token={KEY}' 
  df = requests.get(request).json() # single api request via http = slow!
  final_df = final_df.append(
      pd.Series(
          [ticker,
           df['latestPrice'],
           df['marketCap'],
           'n/a'           
          ],
          index = ct),
      ignore_index = True
  )

#del fdf
final_df

,ticker,close price,market cap,buy qty
0,A,130.807,39909415616,n/a
1,AAL,24.190,15487892066,n/a
2,AAP,193.170,12276682449,n/a
3,AAPL,124.720,2161141062212,n/a
4,ABBV,109.600,200131344644,n/a


In [27]:
def chunks(lst, n): # function to divide ticker list for multiple api request
  for i in range(0, len(lst),n):
    yield lst[i:i + n]

tgroups = list(chunks(tlist['Ticker'], 100))
tstrings = []
for i in range(0, len(tgroups)):
  # print(tgroups[i])
  tstrings.append(','.join(tgroups[i]))
  # print(tstrings[i])

final_df = pd.DataFrame(columns = ct)
for tstring in tstrings:
  batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={tstring}&types=quote&token={KEY}'
  data = requests.get(batch_api_call_url).json() 
  #print(data.status_code) # for testing before applying .json(), should be 200
  for ticker in tstring.split(','):
    #print(ticker)
    final_df = final_df.append(
        pd.Series([
                   ticker,
                   data[ticker]['quote']['latestPrice'],
                   data[ticker]['quote']['marketCap'],
                   'n/a'
        ],
        index = ct ),
      ignore_index = True
    ) # multiple parsing instead of single request
  final_df


In [28]:
final_df

,ticker,close price,market cap,buy qty
0,A,128.566,40156722082,n/a
1,AAL,24.657,15762755512,n/a
2,AAP,191.880,12420617139,n/a
3,AAPL,124.470,2097292378113,n/a
4,ABBV,109.800,197151029590,n/a
...,...,...,...,...
500,YUM,114.437,33633276709,n/a
501,ZBH,164.200,35401864292,n/a
502,ZBRA,501.610,26679280042,n/a
503,ZION,57.080,9434670134,n/a


In [40]:
# 1:11:00 from here on no longer objective of learning to use api, just finishing the video course
psize = input('set portfolio size:')
try:
  val = float(psize)
  print(val)
except ValueError:
  print('error: must be an integer')
  psize = input('set portfolio size:')
  val = float(psize)

position = val/len(final_df.index)
#print(position)
#AAPL2buy = position/500 # concept of determining qty to buy
#print(math.floor(AAPL2buy))

for i in range(0, len(final_df.index)):
  final_df.loc[i, 'buy qty'] = math.floor(position/final_df.loc[i, 'close price'])

final_df

set portfolio size:1000000
1000000.0


,ticker,close price,market cap,buy qty
0,A,128.566,40156722082,15
1,AAL,24.657,15762755512,80
2,AAP,191.880,12420617139,10
3,AAPL,124.470,2097292378113,15
4,ABBV,109.800,197151029590,18
...,...,...,...,...
500,YUM,114.437,33633276709,17
501,ZBH,164.200,35401864292,12
502,ZBRA,501.610,26679280042,3
503,ZION,57.080,9434670134,34


In [46]:
# 1:25:00 testing xls(x) output
writer = pd.ExcelWriter('output.xlsx')
final_df.to_excel(writer, 'qty2buy', index=False)

In [47]:


"""
final_dataframe = fnl_df
portfolio_size = psize
position_size = position
symbol = ticker
symbol_groups = tgroups
symbol_strings = tstrings
"""

"""
# JavaScript Object Notation
df = pd.read_json(request)
df
# need API key to function!!!
"""

'\n# JavaScript Object Notation\ndf = pd.read_json(request)\ndf\n# need API key to function!!!\n'